In [1]:
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

f = open("../../dataset/sense/dict_sense-keys", 'rb')
dict_sense_keys = pickle.load(f)
f.close()

f = open("../../dataset/sense/dict_word-sense", 'rb')
dict_word_sense = pickle.load(f)
f.close()

f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

with open('/data/aviraj/dataset/raw_preprocess_train','rb') as f:
    data=pickle.load(f)

with open('/data/aviraj/dataset/fulldata_vocab_sense','rb') as f:
    vocab_sense=pickle.load(f)

sense2id = dict((s, i) for i,s in enumerate(vocab_sense))
id2sense = dict((i, s) for i,s in enumerate(vocab_sense))

print(len(vocab_sense))

num_senses = len(vocab_sense)
max_sent_size = 200

46


In [2]:
data_x = []
data_label = []
for i in range(len(data)):
    if not all(np.array(data[i][2])==None) and (len(data[i][1])<=200):
        data_label.append([dict_sense_keys[ss][3] if ss is not None else None for ss in data[i][2]])
        data_x.append(data[i][1])

In [ ]:
data_y = []
for i in range(len(data)):
    if (len(data[i][1])<=200):
        for j in range(len(data[i][2])):
            if data[i][2][j] is not None:
                data_y.append(dict_sense_keys[data[i][2][j]][3])

sense_count = Counter(data_y)
sense_count = sense_count.most_common()
sense_count

[('3:00', 105101),
 ('5:00', 99159),
 ('1:04', 85106),
 ('1:10', 66774),
 ('1:09', 61155),
 ('2:42', 55440),
 ('2:32', 52567),
 ('1:14', 51455),
 ('2:31', 45470),
 ('2:41', 40405),
 ('1:07', 39163),
 ('1:06', 38029),
 ('1:18', 37116),
 ('2:30', 32092),
 ('1:28', 29719),
 ('1:26', 29119),
 ('2:40', 24892),
 ('2:38', 20997),
 ('4:02', 18940),
 ('1:15', 18260),
 ('2:36', 17601),
 ('1:03', 17586),
 ('1:21', 16692),
 ('3:01', 15778),
 ('2:35', 15561),
 ('1:11', 12982),
 ('2:39', 12803),
 ('1:23', 10659),
 ('1:19', 6918),
 ('1:17', 6472),
 ('2:34', 6390),
 ('2:33', 6213),
 ('1:27', 6121),
 ('2:37', 6076),
 ('1:24', 6013),
 ('1:08', 5675),
 ('1:12', 4667),
 ('1:22', 3205),
 ('2:29', 3092),
 ('1:05', 1754),
 ('1:16', 1013),
 ('1:25', 938),
 ('1:20', 928),
 ('1:13', 812),
 ('2:43', 110),
 ('3:44', 35)]

In [ ]:
dict_sense_count = dict(sense_count)

split_label = []
for lab in data_label:
        min_idx = np.argmin([dict_sense_count[lab[i]] if lab[i] is not None else np.inf for i in range(len(lab))  ]) 
        split_label.append(lab[min_idx])

print(len(split_label))

In [ ]:
def data_prepare(x, y):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    ss_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    yy = np.zeros([num_examples,max_sent_size], dtype=int)
    
    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            s = y[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            ss_mask[j][i] = True if s in vocab_sense else False
            yy[j][i] = sense2id[s] if s in vocab_sense else 0
        
    return xx, xx_mask, ss_mask, yy

In [ ]:
index = []
for i in range(len(data_x)):
    index.append(i)

index_train, index_val, label_train_id, label_val_id = train_test_split(index, split_label, train_size=0.8, shuffle=True, stratify=split_label, random_state=0)

In [ ]:
data_x = np.array(data_x)
data_label = np.array(data_label)

x_train = data_x[index_train]
y_train = data_label[index_train]
x_val = data_x[index_val]
y_val = data_label[index_val]

x_id_train, mask_train, sense_mask_train, y_train = data_prepare(x_train, y_train)
x_id_val, mask_val, sense_mask_val, y_val = data_prepare(x_val, y_val)

with open('/data/aviraj/dataset/train_data','wb') as f:
    pickle.dump([x_id_train,mask_train,sense_mask_train,y_train],f)
    
with open('/data/aviraj/dataset/val_data','wb') as f:
    pickle.dump([x_id_val,mask_val,sense_mask_val,y_val],f)
    
print(len(x_id_train)+len(x_id_val))

In [ ]:
list_sample = [('1:19', 10000),
 ('1:17', 10000),
 ('2:34', 10000),
 ('2:33', 10000),
 ('1:27', 10000),
 ('2:37', 8000),
 ('1:24', 8000),
 ('1:08', 8000),
 ('1:12', 7000),
 ('1:22', 5000),
 ('2:29', 5000),
 ('1:05', 3000),
 ('1:16', 3000),
 ('1:25', 3000),
 ('1:20', 3000),
 ('1:13', 2000),
 ('2:43', 1100),
 ('3:44', 1000)]

dict_sample = dict(list_sample)

In [ ]:
sm = RandomOverSampler(ratio=dict_sample)

index_train = np.array(index_train).reshape(-1, 1)
sampled_index, _ = sm.fit_sample(index_train, label_train_id)

print(len(sampled_index_train[0]), len(index_train))

In [ ]:
count = Counter(_)
count = count.most_common()
count

In [ ]:
sampled_index_train = np.array(sampled_index).reshape(1, -1)
sampled_x_train = data_x[sampled_index_train[0]]
sampled_y_train = data_label[sampled_index_train[0]]

print(len(sampled_x_train), len(sampled_y_train))

In [ ]:
sampled_x_id_train, sampled_mask_train, sampled_sense_mask_train, sampled_y_train = data_prepare(sampled_x_train, sampled_y_train)

with open('/data/aviraj/dataset/sampled_train_data','wb') as f:
    pickle.dump([sampled_x_id_train, sampled_mask_train, sampled_sense_mask_train, sampled_y_train],f)